# In-Silico Perturbation

**Helical-ai:** Develop a workflow to simulate knock-up and knock-down experiments for specified
genes. The workflow should integrate biologically relevant noise to mimic real-world
variability and provide flexibility for scaling to multiple genes.

**Titouan:** I chose a simple approach using the AnnData object, as it is commonly used in the Helical package. I built a function that processes count data and allows you to apply either a "knock-down" or "knock-up" effect on a specified list of genes. This means you can increase or decrease the counts of those genes by a specified percentage. To better simulate real-world variability, I also added noise to the data. Finally, I created a separate function to simulate count data, which I used to test my implementation.

## 1) Imports

In [1]:
import numpy as np
from anndata import AnnData

## 2) Functions

In [2]:
def simulate_count_data(n_gene: int, n_samples: int, max_count: int = 512) -> AnnData:
    """
    Simulate count data for gene expression analysis.

    Generates a random count matrix with integer values between 0 and `max_count`, 
    stored in an AnnData object with simulated gene and sample names.

    Parameters
    ----------
    n_gene : int
        Number of genes (rows).
    n_samples : int
        Number of samples or cells (columns).
    max_count : int, optional
        Maximum count value (default is 512).

    Returns
    -------
    AnnData
        An AnnData object containing:
        - `X`: Random count matrix (n_gene x n_samples).
        - `obs`: Sample metadata with 'samples' column.
        - `var`: Gene metadata with 'gene_names' column.
    adata = AnnData(
        X = np.random.randint(0, 512, size=(n_gene, n_samples)),
    )
    """
    adata = AnnData(
        X = np.random.randint(0, 512, size=(n_gene, n_samples)),
    )
    samples = [f"CELL{i:d}" for i in range(adata.n_obs)]
    gene_names = [f"GENE{i:d}" for i in range(adata.n_vars)]
    adata.obs.index = samples
    adata.var.index = gene_names
    adata.obs['samples'] = samples
    adata.var['gene_names'] = gene_names

    return adata

In [3]:
def perturb_genes_counts(
    adata: AnnData,
    gene_names,
    target_genes: list,
    knock_type: str = "knock-down",
    fold_change: float = 0.5,
    noise_level: float = 0.1,
) -> AnnData:
    """
    Simulates knock-up or knock-down of specified genes in an AnnData object with count data.

    Args:
        adata (AnnData): The AnnData object containing the count matrix.
        target_genes (list): List of genes to perturb.
        knock_type (str): "knock-up" or "knock-down".
        fold_change (float): Fold change to apply (e.g., 0.5 for 50% knock-down).
        noise_level (float): Noise level to add to the count data as a proportion of the mean.

    Returns:
        AnnData: Modified AnnData object with perturbed counts.
    """
    perturbed_adata = adata.copy()
    X = perturbed_adata.X

    if not np.issubdtype(X.dtype, np.integer):
        raise ValueError("Count matrix must contain integer values.")

    for gene in target_genes:
        if gene not in adata.var[gene_names].tolist():
            print(f"Gene {gene} not found in the dataset!")
            continue

        gene_idx = perturbed_adata.var[gene_names].tolist().index(gene)
        original_counts = X[:, gene_idx]

        # Apply knock-up or knock-down
        if knock_type == "knock-down":
            perturbed_counts = (original_counts * (1 - fold_change)).astype(int)
        elif knock_type == "knock-up":
            perturbed_counts = (original_counts * (1 + fold_change)).astype(int)
        else:
            raise ValueError("knock_type must be 'knock-up' or 'knock-down'")

        # Add Poisson noise to simulate variability in count data
        noise = np.random.poisson(lam=noise_level * np.maximum(perturbed_counts, 1))
        perturbed_counts += noise

        # Clip to ensure no negative values
        perturbed_counts = np.clip(perturbed_counts, 0, None).astype(int)
        X[:, gene_idx] = perturbed_counts

    # Return the updated AnnData object
    perturbed_adata.X = X
    return perturbed_adata

## 3) Testing

In [4]:
adata = simulate_count_data(10, 20)
adata

AnnData object with n_obs × n_vars = 10 × 20
    obs: 'samples'
    var: 'gene_names'

In [5]:
perturb_genes_counts(adata, "gene_names", ["GENE2", "GENE4"], "knock-up")

AnnData object with n_obs × n_vars = 10 × 20
    obs: 'samples'
    var: 'gene_names'

In [6]:
adata = simulate_count_data(5, 20)
perturbed_adata = perturb_genes_counts(adata, "gene_names", ["GENE5"], knock_type="knock-down", fold_change=0.5, noise_level=0.1)

print("Original Expression for 'GENE5':", adata[:, 'GENE5'].X.flatten())
print("Perturbed Expression for 'GENE5':", perturbed_adata[:, 'GENE5'].X.flatten())

Original Expression for 'GENE5': [484 206 261 427 331]
Perturbed Expression for 'GENE5': [268 112 141 232 180]


Since we applied a knock-down effect, the perturbed count should be lower than the original.